In [1]:
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import pickle

In [2]:
tags_inv = {
    0: '[PAD]',
    1: 'O',
    2: 'B-PER',
    3: 'I-PER',
    4: 'B-ORG',
    5: 'I-ORG',
    6: 'B-LOC',
    7: 'I-LOC'
}

# tags_inv = {
#     0: 'O',
#     1: 'B-PER',
#     2: 'I-PER',
#     3: 'B-ORG',
#     4: 'I-ORG',
#     5: 'B-LOC',
#     6: 'I-LOC'
# }

In [3]:
with open('lengths.npy', 'rb') as f:
    lengths = np.load(f)
lengths_100 = lengths[(lengths != 1) & (lengths <= 100)]

In [4]:
with open('pickle/words_all.pickle', 'rb') as f:
    words_all = pickle.load(f)

In [4]:
# with open('pickle/tags_all.pickle', 'rb') as f:
#     tags_all = pickle.load(f)

TypeError: 'int' object is not iterable

In [4]:
# tags_all_pad = []
# for tag in tqdm(tags_all):
#     tags_all_pad.append(tag+1)

In [5]:
# with open('pickle/tags_all_pad.pickle', 'wb') as f:
#     pickle.dump(tags_all_pad, f)

with open('pickle/tags_all_pad.pickle', 'rb') as f:
    tags_all_pad = pickle.load(f)

In [6]:
# y_100 = tf.keras.utils.pad_sequences(tags_all, maxlen=100, padding='post', value=0, dtype='uint8')
# y_100_pad = tf.keras.utils.pad_sequences(tags_all_pad, maxlen=100, padding='post', value=0, dtype='uint8')

# y = tf.keras.utils.to_categorical(y, num_classes=7, dtype='uint8')

In [7]:
# with open('pickle/y_100_pad.pickle', 'wb') as f:
#     pickle.dump(y_100_pad, f)

with open('pickle/y_100_pad.pickle', 'rb') as f:
    y_100_pad = pickle.load(f)

In [8]:
# # with open('pickle/y_100.pickle', 'wb') as f:
# #     pickle.dump(y_100, f)

# with open('pickle/y_100.pickle', 'rb') as f:
#     y_100 = pickle.load(f)

In [4]:
y_100[0]

array([0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
for i in tqdm(range(len(lengths_100))):
    l = lengths_100[i]
    y_100[i][l:]

In [8]:
y_100.shape

(8150004, 100)

In [5]:
elmo = hub.load(r'C:\Users\Egor\Desktop\python\VKR\elmo_ru-news_wmt11-16_1.5M_steps')

In [6]:
chunk_size = 1000
batch_size = 200
total_size = 100_000

tf.keras.backend.clear_session()
step = 0
for i in tqdm(range(total_size // chunk_size)):
    new_chunk=True
    step_chunk = 0
    for j in tqdm(range(i*chunk_size, i*chunk_size+chunk_size, batch_size), leave=False):
        step += 1
        step_chunk += 1
        sentences = words_all[j:j+batch_size]
        ls = lengths_100[j:j+batch_size]
    
        x_emb = tf.keras.utils.pad_sequences(sentences, maxlen=100, padding='post', value='', dtype=object)
        x_emb = elmo.signatures['tokens'](tokens=tf.cast(x_emb, tf.string),
                                          sequence_len = tf.cast(ls, tf.int32))['elmo']
        if step % 50 == 0:
            tf.keras.backend.clear_session()
            
        if new_chunk:
            x_chunk = np.zeros((chunk_size, 100, 1024), dtype=np.float32)
            new_chunk=False
        
        x_chunk[(step_chunk-1)*batch_size:step_chunk*batch_size] = x_emb.numpy()
    
#     tf.keras.backend.clear_session()
    with open(f'D:/VKR_data/data/x_elmo_emb/X_{(i+1):03d}.npy', 'wb') as f:
        np.save(f, x_chunk)

tf.keras.backend.clear_session()

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
def data_gen(batch_size = 128):
    for i in range(len(lengths_100) // batch_size):
        print(i*batch_size)
        sentences = words_all[i*batch_size:i*batch_size+batch_size]
        tags = y[i*batch_size:i*batch_size+batch_size]
        ls = lengths_100[i*batch_size:i*batch_size+batch_size]
        
        x_emb = tf.keras.utils.pad_sequences(sentences, maxlen=100, padding='post', value='', dtype=object)
        x_emb = elmo.signatures['tokens'](tokens=tf.cast(x_emb, tf.string),
                                          sequence_len = tf.cast(ls, tf.int32))['elmo']
        yield x_emb, tags

In [8]:
dataset = tf.data.Dataset.from_generator(
    data_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, 100, 1024), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 100, 7), dtype=tf.uint8)),
).cache()

In [10]:
for xe, ta in tqdm(dataset.take(4)):
    print(xe.shape, ta.shape)

0it [00:00, ?it/s]

0
(128, 100, 1024) (128, 100, 7)
128
(128, 100, 1024) (128, 100, 7)
256
(128, 100, 1024) (128, 100, 7)
384
(128, 100, 1024) (128, 100, 7)


In [10]:
xe

NameError: name 'xe' is not defined